## Objectifs du notebook :

**L’application utilisée par le banquier nous envoie les informations suivantes :** 

_{
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}_

- Créez un nouveau notebook, dans lequel vous calculerez une probabilité de défaut de remboursement.
- Committez, et envoyez moi le lien pour que je retrouve facilement
- Dites “exo fini” ici-même, quand vous avez fini.

### Import des librairies et connection à l'API BigML

In [31]:
from bigml.api import BigML
from bigml.ensemble import Ensemble
import pandas as pd
# from pandas import read_csv

# api = BigML(project='project/5d94a454eba31d46690001d1')
api = BigML()

### Nouvel input

In [51]:
df_data = {"RevolvingUtilizationOfUnsecuredLines": 0.01703559,
 "NumberOfDependents": 1,
 "DebtRatio": 0,
 "age": 42,
 "NumberOfOpenCreditLinesAndLoans": 6,
 "NumberRealEstateLoansOrLines": 1,
 "NumberOfTime30-59DaysPastDueNotWorse": 1,
 "NumberOfTime60-89DaysPastDueNotWorse": 0,
 "NumberOfTimes90DaysLate": 0,
 "MonthlyIncome": 0}

df = pd.DataFrame([df_data])
df

,DebtRatio,MonthlyIncome,NumberOfDependents,NumberOfOpenCreditLinesAndLoans,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,RevolvingUtilizationOfUnsecuredLines,age
0,0,0,1,6,1,0,0,1,0.017036,42


### Utilisation d'une méthode pour avoir un dataset avec toutes les modifications

In [52]:
from ipynb.fs.full.new_input import change_dataset

In [53]:
df_modify = change_dataset(df)
df_modify

,DebtRatio,MonthlyIncome,NumberOfDependents,NumberOfOpenCreditLinesAndLoans,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,RevolvingUtilizationOfUnsecuredLines,age,IncomePerPerson,isOld,MonthlyDebt,MonthlyBalance,DebtPerPerson,BalancePerPerson,NumberOfTime30-89DaysPastDueNotWorse,NumbersOfOpen-NumberRealEstate
Id,,,,,,,,,,,,,,,,,,
0,0,0,1,6,1,0,0,1,0.017036,42,0.0,0,0,0,0.0,0.0,1,5


### Transformation du dataset en list pour pouvoir lui appliquer un modèle et faire la prédiction

In [93]:
input_data = df_modify.to_dict('index')

input_data

{0: {'DebtRatio': 0,
  'MonthlyIncome': 0,
  'NumberOfDependents': 1,
  'NumberOfOpenCreditLinesAndLoans': 6,
  'NumberOfTime30-59DaysPastDueNotWorse': 1,
  'NumberOfTime60-89DaysPastDueNotWorse': 0,
  'NumberOfTimes90DaysLate': 0,
  'NumberRealEstateLoansOrLines': 1,
  'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
  'age': 42,
  'IncomePerPerson': 0.0,
  'isOld': '0',
  'MonthlyDebt': 0,
  'MonthlyBalance': 0,
  'DebtPerPerson': 0.0,
  'BalancePerPerson': 0.0,
  'NumberOfTime30-89DaysPastDueNotWorse': 1,
  'NumbersOfOpen-NumberRealEstate': 5}}

### Modèle

In [57]:
model_ensemble = Ensemble('ensemble/5dc2d7d3e476847468013866', api=BigML(domain="bigml.io"))

### Prédiction

In [94]:
input_prediction = model_ensemble.predict(input_data, full=True)
input_prediction

{'probability': 0.93283, 'prediction': '0'}